In [2]:
import sys
sys.path.append("..")

from molsberry.core import Pipeline, InputBlock, OutputBlock
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData

from molsberry.modules.mopac import MOPACMozymeConfig
from molsberry.modules.mopac import (
    MOPACSinglePointCalculator, MOPACPLInteractionCalculator,
    MOPACLigandOptimizer
)

In [3]:
[x for x in type(OutputBlock([])).__mro__][0]()

TypeError: OutputBlock.__init__() missing 1 required positional argument: 'keys'

In [5]:
class BindingEnergyCalcPipe(Pipeline):
    name = "bindenpipe"
    display_name = "Binding Energy Calculation Pipeline"
    def build(self):
        config = MOPACMozymeConfig()
        config.keywords.extend(["PM6-D3H4X", "EPS=78.5", "VDW(C=1.821;N=1.904;O=1.682;H=0.828)", "NOMM"])
        self.add_block(InputBlock(["ligand", "protein"]), "input")
        self.add_block(MOPACPLInteractionCalculator(config), "intercalc")
        self.add_block(MOPACLigandOptimizer(config, "LBFGS"), "ligopt")
        self.add_block(OutputBlock(["e_lig_init", "e_lig_opt", "e_interaction"]), "output")
        self.add_connection("input", "ligand", "intercalc", "ligands")
        self.add_connection("input", "protein", "intercalc", "proteins")
        self.add_connection("input", "ligand", "ligopt", "ligands")
        self.add_connection("intercalc", "e_interaction", "output", "e_interaction")
        self.add_connection("ligopt", "e_init", "output", "e_lig_init")
        self.add_connection("ligopt", "e_final", "output", "e_lig_opt")

In [6]:
ligdata = LigandData(SDFPathRep("lig.sdf"))
pocdata = ProteinData(PDBPathRep("poc.pdb"))

In [7]:
pipeline = BindingEnergyCalcPipe(base_dir="./example1_out")
out = pipeline.execute(ligand=ligdata, protein=pocdata)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>   STARTED: Binding Energy Calculation Pipeline    >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (intercalc) MOPAC Protein Ligand Interaction Calculator]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f5d3cccad70> <class 'molsberry.core.data.molecules.LigandData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>
<class 'molsberry.core.data.molecules.ProteinData'>
<molsberry.core.data.molecules.ProteinData object at 0x7f5d3cccb7f0> <class 'molsberry.core.data.molecules.ProteinData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>

[Running Pipe Block: (ligopt) MOPAC Ligand Optimizer]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f5d3cccad70> <class 'molsberry.core.data.molecules.LigandData'>
[<class 'molsberry.modules.mopac.representations.M

In [8]:
{k: v.get_representation_content()[0] for k, v in out.items()}

{'e_interaction': 6.563009999999849,
 'e_lig_init': -27.56031,
 'e_lig_opt': -30.01346}